In [8]:
pip install sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.5 MB/s eta 0:00:00


In [29]:
import pandas as pd

df = pd.read_csv("courses_detailed.csv")
ALL_CODES = set(df["course_code"].dropna().astype(str))


print(df.columns)
print(len(df), "rows")
df.head()


Index(['program_name', 'req_num', 'course_code', 'title', 'credits', 'offered',
       'description', 'grading', 'repeatable', 'prerequisites', 'extra_blocks',
       'detail_url'],
      dtype='object')
3166 rows


,program_name,req_num,course_code,title,credits,offered,description,grading,repeatable,prerequisites,extra_blocks,detail_url
0,Advanced Mathematical Methods (for Students in...,1,MATH-UA 140,Linear Algebra,(4 Credits),"Fall, Spring, and Summer terms",Systems of linear equations. Gaussian eliminat...,CAS Graded,No,NaN,NaN,https://bulletins.nyu.edu/search/?P=MATH-UA%20140
1,Advanced Mathematical Methods (for Students in...,1,MATH-UA 148,Honors Linear Algebra,(4 Credits),NaN,This honors section of Linear Algebra is a pro...,CAS Graded,No,NaN,NaN,https://bulletins.nyu.edu/search/?P=MATH-UA%20148
2,Advanced Mathematical Methods (for Students in...,2,MATH-UA 352,Numerical Analysis,(4 Credits),Spring,Formerly numbered MATH-UA 252; the content has...,CAS Graded,No,NaN,NaN,https://bulletins.nyu.edu/search/?P=MATH-UA%20352
3,Advanced Mathematical Methods (for Students in...,2,MATH-UA 358,Honors Numerical Analysis,(4 Credits),NaN,Formerly numbered MATH-UA 258; the content has...,CAS Graded,No,NaN,NaN,https://bulletins.nyu.edu/search/?P=MATH-UA%20358
4,Advanced Mathematical Methods (for Students in...,2,MATH-GA 2010,Numerical Methods I,(3 Credits),Fall,This course is part of a two-course series mea...,GSAS Graded,No,NaN,NaN,https://bulletins.nyu.edu/search/?P=MATH-GA%20...


In [12]:
docs = []
metadata = []

for _, row in df.iterrows():
    doc = f"""
Program: {row['program_name']}
Requirement group: {row['req_num']}

Course: {row['course_code']} - {row['title']}
Credits: {row['credits']}
Typically offered: {row['offered']}
Prerequisites: {row['prerequisites']}
Repeatable: {row['repeatable']}
Grading: {row['grading']}

Description:
{row['description']}

Extra:
{row['extra_blocks']}
""".strip()

    docs.append(doc)
    metadata.append(row.to_dict())

print("Built", len(docs), "course docs.")


Built 3166 course docs.


In [13]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embeddings = embedder.encode(docs, convert_to_numpy=True).astype("float32")

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

print("Index contains", index.ntotal, "vectors.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Index contains 3166 vectors.


In [14]:
def retrieve_courses(query: str, k: int = 5):
    q_emb = embedder.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(q_emb, k)

    results = []
    for dist, idx in zip(distances[0], indices[0]):
        if idx == -1:
            continue
        results.append({
            "score": float(dist),
            "text": docs[idx],
            "meta": metadata[idx],   # original row as dict
        })
    return results


In [38]:
results = retrieve_courses(
    "introductory data science course with no prerequisites and 4 credits",
    k=5
)

for r in results:
    m = r["meta"]
    print(f"Score: {r['score']:.4f}")
    print(f"{m['course_code']} - {m['title']}  ({m['credits']})")
    print("Program:", m["program_name"])
    print("Prereq:", m["prerequisites"])
    print("Offered:", m["offered"])
    print("Detail URL:", m["detail_url"])
    print("-" * 80)


Score: 0.4533
DS-UA 301 - Advanced Topics in Data Science  ((4 Credits))
Program: Data Science (Minor)
Prereq: DS-UA 112 and ( MATH-UA 185 or MATH-UA 334 or MA-UY 2224 as co-requisites ) and ( CSCI-UA 473 as a co-requisite ) and restricted to Majors/Minors.
Offered: Fall and Spring
Detail URL: https://bulletins.nyu.edu/search/?P=DS-UA%20301
--------------------------------------------------------------------------------
Score: 0.4665
DS-UA 301 - Advanced Topics in Data Science  ((4 Credits))
Program: Data Science (BA)
Prereq: DS-UA 112 and ( MATH-UA 185 or MATH-UA 334 or MA-UY 2224 as co-requisites ) and ( CSCI-UA 473 as a co-requisite ) and restricted to Majors/Minors.
Offered: Fall and Spring
Detail URL: https://bulletins.nyu.edu/search/?P=DS-UA%20301
--------------------------------------------------------------------------------
Score: 0.4756
DS-UA 301 - Advanced Topics in Data Science  ((4 Credits))
Program: Computer and Data Science (BA)
Prereq: DS-UA 112 and ( MATH-UA 185 or MAT

In [30]:
def build_prompt(query: str, retrieved):
    context_parts = []
    allowed_codes = set()

    for i, item in enumerate(retrieved, start=1):
        text = item["text"]
        meta = item["meta"]
        code = str(meta.get("course_code", "")).strip()
        if code:
            allowed_codes.add(code)
        context_parts.append(f"[COURSE {i}]\n{text}")

    context = "\n\n".join(context_parts)
    allowed_codes_list = sorted(allowed_codes)

    prompt = f"""
You are an assistant helping a CAS student plan their NYU courses.

CONTEXT COURSES (you may only use these course codes):
{", ".join(allowed_codes_list)}

HARD RULES (you MUST follow all of them):
- Use ONLY courses present in the context above.
- You may NOT invent new course codes. If a course is not in the context,
  use a placeholder WITHOUT a code, like:
  - Course Code: "-"   Course Title: "Core humanities elective"
  - Course Code: "-"   Course Title: "Free elective"
- If a key course is clearly missing (e.g. a beginner Japanese class or a
  specific Data Science requirement), you must:
  - Use a placeholder without a code, and
  - State in the Notes that the exact course is missing from the context.
- Respect all prerequisites and co-requisites mentioned in the context.
- Put lower-level / introductory courses earlier when possible.
- Keep each semester between 14 and 18 credits (aim for 15–16).

OUTPUT FORMAT:
- Return ONLY a markdown table with columns:
  | Year | Semester | Course Code | Course Title | Credits | Notes |
- Do not write anything before or after the table.
- One row per course. Include all 8 semesters (4 years × Fall/Spring).

Context:
{context}

Student request:
{query}
""".strip()
    return prompt


In [31]:
import re

def clean_plan_markdown(plan_md: str) -> str:
    """
    Post-process the markdown table:
    - If a course code is not in ALL_CODES, replace it with "-" and
      keep the title as a placeholder.
    """
    lines = plan_md.strip().splitlines()
    if not lines:
        return plan_md

    # Assume first line is header, maybe second is separator
    header = lines[0]
    sep = lines[1] if len(lines) > 1 and set(lines[1]) <= set("|- ") else None
    body = lines[2:] if sep else lines[1:]

    cleaned_rows = []

    for line in body:
        if not line.strip().startswith("|"):
            continue
        cells = [c.strip() for c in line.strip().strip("|").split("|")]
        if len(cells) < 6:
            continue

        year, sem, code, title, credits, notes = cells[:6]

        # If the code is not a real catalog code (according to df), treat as placeholder
        if code not in ALL_CODES and code not in ("-", ""):
            if not title:
                title = "Free elective"
            code = "-"
            if notes:
                notes = notes + " (original code not in catalog context)"
            else:
                notes = "Original code not in catalog context"

        cleaned_rows.append(f"| {year} | {sem} | {code} | {title} | {credits} | {notes} |")

    out = [header]
    if sep:
        out.append(sep)
    out.extend(cleaned_rows)
    return "\n".join(out)


In [32]:
def answer_query(query: str, k: int = 5, model: str = "gpt-4.1-mini") -> str:
    retrieved = retrieve_courses(query, k=k)
    prompt = build_prompt(query, retrieved)
    raw = call_llm(prompt, model=model)
    cleaned = clean_plan_markdown(raw)
    return cleaned


In [18]:
def answer_query_with_retrieved(query: str, retrieved, model: str = "gpt-4.1-mini") -> str:
    prompt = build_prompt(query, retrieved)
    return call_llm(prompt, model=model)


In [33]:
from google.colab import userdata
from openai import OpenAI

api_key = userdata.get("OPENAI_API_KEY")
assert api_key, "No API key found in Colab userdata under 'OPENAI_API_KEY'"

client = OpenAI(api_key=api_key)


In [37]:
plan_query = """
I am an incoming first-year student planning to major in Data Science.
Assume I start in Fall 2025, want to take about 15–16 credits per semester,
and finish in 4 years.

Using the course information in the context, propose a tentative 8-semester
plan with course codes and titles. Try to:

- Put introductory/1000-level courses in the first year,
- Respect listed prerequisites when possible,
- Mix required/core DS courses with reasonable electives,
- Leave placeholders like 'Core humanities elective' where the catalog does not give enough detail.

Output the plan as a table with columns: Year, Semester, Course Code, Course Title, Credits, Notes.
"""

print(answer_query(plan_query, k=40))


| Year | Semester | Course Code | Course Title                     | Credits | Notes                                    |
|------|----------|-------------|---------------------------------|---------|------------------------------------------|
| 1 | Fall | CSCI-UA 2 | Introduction to Computer Programming (No Prior Experience) | 4 | Intro programming, no prior experience |
| 1 | Fall | DS-UA 111 | Principles of Data Science I | 4 | Foundational data science and Python |
| 1 | Fall | - | Core humanities elective | 4 | Placeholder for humanities requirement |
| 1 | Fall | - | Free elective | 3 | To reach 15 credits |
| 1 | Spring | CSCI-UA 101 | Intro to Computer Science | 4 | Requires CSCI-UA 2 or equivalent |
| 1 | Spring | MATH-UA 120 | Discrete Mathematics | 4 | Math foundation for CS/DS |
| 1 | Spring | - | Core humanities elective | 4 | Placeholder for humanities requirement |
| 1 | Spring | - | Free elective | 3 | To reach 15 credits |
| 2 | Fall | CSCI-UA 102 | Data Structures | 4 

In [36]:
plan_query = """
I am an incoming first-year student at NYU planning to major in Data Science.
Assume I start in Fall 2025, want to take about 15–16 credits per semester,
and finish in 4 years.

I want to spend one semester studying abroad in my 3rd year (for example,
Fall of junior year). During that semester, I will mainly take electives
or flexible requirements, not heavy prerequisite chains.

Using the course information in the context, propose a tentative 8-semester
plan with course codes and titles. Try to:

- Put introductory/1000-level courses in the first year.
- Respect listed prerequisites when possible.
- Ensure key prerequisite chains are completed BEFORE the study-abroad semester.
- Use the study-abroad semester mostly for electives or core requirements
  that do not break prerequisites.
- Leave placeholders like 'Core humanities elective' or 'Study abroad elective'
  where the catalog does not give enough detail.

Output the plan as a table with columns:
Year, Semester, Course Code, Course Title, Credits, Notes.
"""

print(answer_query(plan_query, k=40))


| Year | Semester  | Course Code | Course Title                     | Credits | Notes                                           |
|-------|------------|-------------|---------------------------------|---------|-------------------------------------------------|
| 1 | Fall | PHYS-UA 11 | General Physics I | 5 | Requires MATH-UA 121 or equivalent |
| 1 | Fall | PSYCH-UA 10 | Statistics for The Behavioral Sciences | 4 | Introductory statistics, no prerequisite |
| 1 | Fall | EG-UY 1004 | Introduction to Engineering and Design | 4 | Introductory engineering course |
| 1 | Fall | - | Core humanities elective | 3 | Placeholder for humanities elective |
| 1 | Spring | COLIT-UA 116 | Approaching Comparative Literature | 4 | Intro to Comp Lit, offered spring only |
| 1 | Spring | GERM-UA 3 | Intermediate German I | 4 | Prerequisite: GERM-UA 2 or equivalent |
| 1 | Spring | ECON-UA 18 | Statistics (P) | 4 | Requires math prereq, complements statistics skill |
| 1 | Spring | - | Core humanities el

In [ ]:
plan_query = """
I am an incoming first-year student at NYU planning to major in Data Science.

Constraints:
- I have 12 IB credits already, so you can assume that some core/elective requirements
  are partially fulfilled. I don't want a super heavy load of random core classes early.
- I want to study Japanese starting from beginner level and continue it for multiple semesters.
- I want to spend one semester abroad (for example, Fall of my third year).
- I'd like to average about 15–16 credits per semester and finish in 4 years.

Using ONLY the course information in the context, propose a tentative 8-semester plan with
course codes and titles. Try to:
- Put introductory/1000-level and fundamental courses in the first year.
- Respect prerequisites where they are listed (e.g. complete math/programming before advanced DS).
- Ensure important prerequisite chains are mostly completed before the study-abroad semester.
- Use the study-abroad semester mostly for electives or flexible requirements.
- Include Japanese language courses where appropriate.
- Leave placeholders like "Core humanities elective" or "Study abroad elective" for requirements
  that are not specified in the course text.

Output the plan as a table with columns:
Year, Semester, Course Code, Course Title, Credits, Notes.
"""

print(answer_query(plan_query, k=60))


In [25]:
plan_query = """
I am an incoming first-year student at NYU in CAS, planning to double major in
Mathematics and Data Science.

Constraints:
- I have 12 IB credits already, so assume some core/elective requirements are fulfilled.
- I want to start Japanese from beginner level and continue it for several semesters.
- I want to spend one semester abroad (Fall of my third year), and during that term
  I prefer to take flexible electives or core courses rather than tight prerequisite chains.
- I want to finish both majors in 4 years, averaging about 15–16 credits per semester.

Using ONLY the course information in the context, propose a tentative 8-semester
plan with course codes and titles. Try to:
- Put introductory programming and lower-division math (Calculus I–III, Linear Algebra)
  in the first two years.
- Schedule key courses for the Math and Data Science majors (e.g. Real Analysis,
  Abstract Algebra, probability and statistics, machine learning, data science capstone, etc.).
- Respect prerequisites as described in the course descriptions whenever possible.
- Ensure that important prerequisite chains are mostly completed before the study-abroad term.
- Include Japanese language courses where appropriate.
- Use the study-abroad semester mostly for electives or core requirements that do not break prerequisites.
- Leave placeholders like "Core humanities elective", "Study abroad elective", or
  "Free elective" where the catalog does not give enough detail.

Output the plan as a table with columns:
Year, Semester, Course Code, Course Title, Credits, Notes.
"""

print(answer_query(plan_query, k=80))


| Year | Semester     | Course Code | Course Title                  | Credits | Notes                                                                                      |
|-------|--------------|-------------|-------------------------------|---------|--------------------------------------------------------------------------------------------|
| 1     | Fall         | MATH-UA 121 | Calculus I                    | 4       | Foundational math for both majors; no prerequisites                                       |
| 1     | Fall         | CSCI-UA 2   | Introduction to Computer Programming (No Prior Experience) | 4       | Intro programming for Data Science; no prior programming experience required              |
| 1     | Fall         | EAST-UA 248 | Beginner Japanese I (assumed prerequisite for EAST-UA 249) | Not listed | Not in context; but EAST-UA 249 prerequisite is EAST-UA 248 with minimum grade C-. Assuming beginner course for Japanese |
| 1     | Fall         | Core humanities e

In [23]:
plan_query = """
You are designing a four-year, 8-semester course plan for a first-year CAS student
majoring in Data Science at NYU.

Student constraints:
- Aiming for ~15–16 credits per semester.
- Has 12 IB credits, so some core / gen-ed requirements are already satisfied.
- Wants a standard 4-year graduation timeline.

Using ONLY the courses and information in the context, produce a detailed 8-semester
plan that:

1. Puts introductory / 1000-level and foundational courses (intro programming,
   intro data science, basic calculus / linear algebra, probability / statistics)
   in the first 3–4 semesters.
2. Respects all listed prerequisites and co-requisites whenever possible.
3. Schedules key Data Science courses (e.g. intro DS, statistics/probability,
   machine learning / data mining, core DS requirements, and a capstone) in a
   sensible order.
4. Keeps each semester between 14 and 18 credits, aiming for 15–16.
5. Includes placeholders like “Core humanities elective”, “Free elective”, or
   “Minor elective” where the catalog does not specify exact courses.
6. Clearly notes when a course is an assumption based on typical structure but
   is not explicitly described in the context.

OUTPUT FORMAT (very important):
Return ONLY a markdown table with the following columns:

| Year | Semester | Course Code | Course Title | Credits | Notes |

- Include one row per course (8 semesters total).
- Do NOT include any text outside of the table.
"""


In [35]:
!pip install gradio -q

import gradio as gr
import traceback

def rag_app(question: str, top_k: int):
    question = question.strip()
    # Empty-question case
    if not question:
        header = [
            "Score (lower = more similar)",
            "Course code",
            "Title",
            "Credits",
            "Offered",
            "Prerequisites",
            "Program",
            "Detail URL",
        ]
        return "Please enter a question about your plan or courses.", []

    try:
        # 1) Retrieve courses from FAISS
        retrieved = retrieve_courses(question, k=int(top_k))

        # 2) Get LLM answer using your existing answer_query()
        answer = answer_query(question, k=int(top_k))

        # 3) Build a *plain* list-of-lists table (no pandas, no numpy types)
        rows = []
        for item in retrieved:
            m = item["meta"]  # original row from the CSV
            rows.append([
                float(item["score"]),
                str(m.get("course_code", "")),
                str(m.get("title", "")),
                str(m.get("credits", "")),
                str(m.get("offered", "")),
                str(m.get("prerequisites", "")),
                str(m.get("program_name", "")),
                str(m.get("detail_url", "")),
            ])

        return answer, rows

    except Exception as e:

        tb = traceback.format_exc()
        msg = f"⚠️ Python error: {e}\n\n```python\n{tb}\n```"
        return msg, []


headers = [
    "Score (lower = more similar)",
    "Course code",
    "Title",
    "Credits",
    "Offered",
    "Prerequisites",
    "Program",
    "Detail URL",
]

demo = gr.Interface(
    fn=rag_app,
    inputs=[
        gr.Textbox(
            label="Your question",
            lines=6,
            placeholder=(
                "Example: I am an incoming first-year CAS student planning to major in "
                "Data Science. I have 12 IB credits and want about 15–16 credits per "
                "semester. Propose an 8-semester plan."
            ),
        ),
        gr.Slider(
            minimum=1,
            maximum=80,
            step=1,
            value=20,
            label="Top-K courses to retrieve",
        ),
    ],
    outputs=[
        gr.Markdown(label="RAG Answer"),
        gr.Dataframe(
            headers=headers,
            label="Retrieved courses used as context",
        ),
    ],
    title="NYU Course RAG Planner",
    description=(
        "Ask questions about course planning, prerequisites, and degree paths. "
        "The system retrieves relevant NYU courses with FAISS + sentence-transformers "
        "and then uses GPT-4.1-mini to generate an answer based ONLY on that context."
    ),
)

# debug=True -> traceback will print in the Colab cell if anything goes wrong
demo.launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://26cd7cf3b9f7875ee9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7863 <> https://26cd7cf3b9f7875ee9.gradio.live


In [ ]:
df = df
docs = docs
metadata = metadata
embedder = embedder
embeddings = embeddings
index = index

def rebuild_index_from_df(df_new: pd.DataFrame):
    """
    Rebuild docs, metadata, embeddings, and FAISS index
    from a new course DataFrame.
    """
    global df, docs, metadata, embeddings, index, embedder

    df = df_new.copy()

    new_docs = []
    new_meta = []

    for _, row in df.iterrows():
        doc = f"""
Program: {row.get('program_name', '')}
Requirement group: {row.get('req_num', '')}

Course: {row.get('course_code', '')} - {row.get('title', '')}
Credits: {row.get('credits', '')}
Typically offered: {row.get('offered', '')}
Prerequisites: {row.get('prerequisites', '')}
Repeatable: {row.get('repeatable', '')}
Grading: {row.get('grading', '')}

Description:
{row.get('description', '')}

Extra:
{row.get('extra_blocks', '')}
""".strip()

        new_docs.append(doc)
        new_meta.append(row.to_dict())

    docs = new_docs
    metadata = new_meta

    print("Encoding", len(docs), "course docs...")
    embeddings = embedder.encode(docs, convert_to_numpy=True).astype("float32")
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    print("New index contains", index.ntotal, "vectors.")


In [ ]:
import io
import gradio as gr

def load_new_catalog(file):
    """
    Gradio callback: load a new CSV, rebuild index, and show a preview.
    """
    if file is None:
        return "No file uploaded yet.", pd.DataFrame()

    try:
        # Read CSV from the uploaded file
        df_new = pd.read_csv(file.name)

        # Rebuild everything
        rebuild_index_from_df(df_new)

        # Small preview
        preview = df_new.head(10)

        msg = (
            f"✅ Loaded new catalog with {len(df_new)} rows.\n\n"
            "The RAG index has been rebuilt. New queries will use this data."
        )
        return msg, preview

    except Exception as e:
        return f"❌ Error loading file: {e}", pd.DataFrame()


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# NYU Course RAG Planner")

    with gr.Tab("Ask the RAG"):
        question = gr.Textbox(
            label="Your question",
            lines=6,
            placeholder=(
                "Example: I am an incoming first-year CAS student planning to major in "
                "Data Science, with 12 IB credits. Propose an 8-semester plan with ~15–16 "
                "credits per semester."
            ),
        )
        top_k = gr.Slider(
            minimum=1,
            maximum=80,
            step=1,
            value=20,
            label="Top-K courses to retrieve",
        )
        answer = gr.Markdown(label="RAG Answer")
        sources_df = gr.Dataframe(label="Retrieved courses used as context")

        ask_btn = gr.Button("Ask RAG")
        ask_btn.click(
            fn=rag_app,  # the function you already wrote
            inputs=[question, top_k],
            outputs=[answer, sources_df],
        )

    with gr.Tab("Upload / Update Catalog"):
        gr.Markdown(
            "Upload a **CSV of courses** to replace the current catalog.\n\n"
            "- File must have the same columns as `courses_detailed.csv` "
            "(`course_code`, `title`, `credits`, `offered`, `prerequisites`, etc.)."
        )

        file_input = gr.File(label="Course catalog CSV", file_types=[".csv"])
        load_btn = gr.Button("Load and rebuild index")

        status_md = gr.Markdown()
        preview_df = gr.Dataframe(label="Preview of new catalog")

        load_btn.click(
            fn=load_new_catalog,
            inputs=file_input,
            outputs=[status_md, preview_df],
        )

demo.launch(share=True, debug=True)
